In [1]:
import numpy as np
import os, sys
from skimage import io
import matplotlib
from pprint import pprint
from matplotlib import pyplot as plt
from tqdm import tqdm
import neuroglancer
import cv2

#from PIL import Image
#Image.MAX_IMAGE_PIXELS = None

In [4]:
HOME = os.path.expanduser("~")
PATH = os.path.join(HOME, 'programming/pipeline_utility/src')
sys.path.append(PATH)
from abakit.lib.FileLocationManager import DATA_PATH, ROOT_DIR
from abakit.lib.utilities_atlas import  singular_structures, average_shape, mirror_volume_v2

In [5]:
atlas_name = 'atlasV8'

In [ ]:
structure = 'SC'
color = 100
origins = []
volumes = []


filepath = os.path.join(VOLUME_PATH, f'{structure}.npy')
#volume = io.imread(midfilepath)
volume = np.load(filepath)
volume = volume * color
volume = volume.astype(np.uint8)
print(volume.shape, volume.dtype, np.amin(volume), np.amax(volume), np.mean(volume))

In [ ]:
#volume = volume.reshape(volume.shape[0], volume.shape[1], volume.shape[2])
arr = volume.copy()
print('volume shape', arr.shape, arr.dtype, np.unique(arr, return_counts=False))

In [ ]:
#volume[volume > 0] = 255
#volume[volume < 1] = 0
flat = arr.flatten()
fig = plt.figure()
plt.rcParams['figure.figsize'] = [10, 6]
plt.hist(flat, bins=20, color='b')
plt.style.use('ggplot')
plt.yscale('log')
plt.grid(axis='y', alpha=0.75)
plt.xlabel('Value')
plt.ylabel('Frequency')
plt.title(f' {arr.dtype}')

In [ ]:
arr = volume.copy()
#arr = np.rot90(volume, axes=(2, 1))
#arr = np.rot90(arr, 3)
#arr = np.flip(arr, axis=1)    
print(arr.shape)
start = 20
end = start + 10
plt.style.use('classic')
for i in range(start, end):
    plt.title('Z {}'.format(i))
    plt.imshow(arr[:,:,i], cmap="cool")
    plt.show()

In [ ]:
dimensions = neuroglancer.CoordinateSpace(
    names=['x', 'y', 'z'],
    units='nm',
    scales=[1000,1000,1000])

viewer = neuroglancer.Viewer()
with viewer.txn() as s:
    s.dimensions = dimensions
    s.layers['Altas'] = neuroglancer.ImageLayer(
        source=neuroglancer.LocalVolume(
            data=volume,
            dimensions=dimensions,
            voxel_offset=(0, 0, 0),
        ),
        shader="""
#uicontrol float min slider(min=0, max=1, default=0)
#uicontrol float max slider(min=0, max=1, default=1)
#uicontrol float invert slider(min=0, max=1, default=0, step=1)
#uicontrol float brightness slider(min=-1, max=1)
#uicontrol float contrast slider(min=-3, max=3, step=0.01)
void main() {
  float pix_val = toNormalized(getDataValue());
  if(pix_val < min){
  	pix_val = 0.0;
  }
  if(pix_val > max){
    pix_val = 1.0;
  }

  if(invert==1.0){
  	  emitGrayscale((1.0 -(pix_val - brightness)) *
       exp(contrast));
  }
  else{
    emitGrayscale((pix_val + brightness) *
                  exp(contrast));
  }

}

""")
print(viewer)

In [ ]:
files